In [1]:
import folium
from geopy.distance import geodesic
from geopy.point import Point
import numpy as np

In [2]:
# Function to add vision cone
def add_vision_cone(map_obj, lat, lon, azimuth, opening_angle, dist_km, color, label):
    center = [lat, lon]
    start_angle = (azimuth - opening_angle / 2) % 360
    end_angle = (azimuth + opening_angle / 2) % 360
    num_points = 50
    
    # Generate angles for cone edges
    if start_angle < end_angle:
        angles = np.linspace(start_angle, end_angle, num_points)
    else:
        angles = np.concatenate((np.linspace(start_angle, 360, num_points // 2), 
                                 np.linspace(0, end_angle, num_points // 2)))
    
    # Generate points along the edge of the cone
    edge_points = [
        (geodesic(kilometers=dist_km).destination(Point(lat, lon), angle).latitude,
         geodesic(kilometers=dist_km).destination(Point(lat, lon), angle).longitude)
        for angle in angles
    ]
    
    # Define the polygon points by connecting the center to the edge points
    polygon_points = [center, *edge_points, center]

    # Add polygon to map
    folium.Polygon(
        locations=polygon_points,
        color=color,
        fill=True,
        fill_opacity=0.4,
        popup=label
    ).add_to(map_obj)




def add_cam(map_obj, cam_index, initial_azimuth_cam, rotation_angle, overlap, site_lat, site_lon, fov, dist_km, camera_colors):

    start_azimuth_cam = (initial_azimuth_cam + 180 +fov/2 -(4*fov - 3*overlap -180) + overlap/2)%360
    # Generate azimuth angles for Camera 1 and add vision cones
    azimuths_cam = []
    for i in range(2,6):
        azimuths_cam.append((start_azimuth_cam + i*rotation_angle ) % 360)
    for i, azimuth in enumerate(azimuths_cam):
        add_vision_cone(
            map_obj, site_lat, site_lon, azimuth=azimuth,
            opening_angle=fov, dist_km=dist_km,
            color=camera_colors[i%len(camera_colors)], label=f"{cam_index}: {azimuth}°"
        )

In [3]:
# Parameters
site_lat, site_lon = -33.45176001443013, -71.18499516451242  # Center coordinates
fov = 54.2                          # Field of view of each camera (degrees)
dist_km = 20                        # Visualization distance (km)
overlap = 6                         # Overlap between cam positions
initial_azimuth_cam1 = 90           # Initial azimuth for Camera 1
initial_azimuth_cam2 = 257          # Initial azimuth for Camera 2
rotation_angle = fov - overlap           # Rotation angle with overlap considered
camera1_colors = ['#6600cc', '#9900cc', '#cc33ff', '#e066ff']  # Colors for Camera 1
camera2_colors = ['#0033cc', '#3366cc', '#6699ff', '#99ccff']  # Colors for Camera 2


In [4]:
# Create map
map_obj = folium.Map(location=[site_lat, site_lon], zoom_start=11)

# Cam1
add_cam(map_obj, "Cam 1", initial_azimuth_cam1, rotation_angle, overlap, site_lat, site_lon, fov, dist_km, camera1_colors)

# Cam2
add_cam(map_obj, "Cam 2", initial_azimuth_cam2, rotation_angle, overlap, site_lat, site_lon, fov, dist_km, camera2_colors)

In [5]:
# Display the map
map_obj

In [8]:
# Function to add all vision cones for one camera
def add_camera_cones(map_obj, cam_label, lat, lon, azimuth_list, fov, dist_km, colors):
    for i, azimuth in enumerate(azimuth_list):
        add_vision_cone(
            map_obj, lat, lon, azimuth=azimuth,
            opening_angle=fov, dist_km=dist_km,
            color=colors[i % len(colors)],
            label=f"{cam_label} - {azimuth:.2f}°"
        )


# Example usage
site_lat = 44.200819722774405
site_lon = 3.1407973740079225
fov = 54.2
dist_km = 20

# Example azimuths for two cameras
cam1_azimuths = [33.588, 84.926, 136.264, 187.602]
cam2_azimuths = [216.300, 267.638, 318.976, 10.314]

# Example colors
cam1_colors = ['#6600cc', '#9900cc', '#cc33ff', '#e066ff']
cam2_colors = ['#0033cc', '#3366cc', '#6699ff', '#99ccff']

# Create map
map_obj = folium.Map(location=[site_lat, site_lon], zoom_start=11)

# Add cones for each camera
add_camera_cones(map_obj, "Cam 1", site_lat, site_lon, cam1_azimuths, fov, dist_km, cam1_colors)
add_camera_cones(map_obj, "Cam 2", site_lat, site_lon, cam2_azimuths, fov, dist_km, cam2_colors)


In [9]:
# Display the map
map_obj

In [10]:
# Example usage
site_lat = 44.200819722774405
site_lon = 3.1407973740079225
fov = 54.2
dist_km = 20

# Example azimuths for two cameras
cam1_azimuths = [213, 264, 315]
cam2_azimuths = [357, 48, 100]

# Example colors
cam1_colors = ['#6600cc', '#9900cc', '#cc33ff', '#e066ff']
cam2_colors = ['#0033cc', '#3366cc', '#6699ff', '#99ccff']

# Create map
map_obj = folium.Map(location=[site_lat, site_lon], zoom_start=11)

# Add cones for each camera
add_camera_cones(map_obj, "Cam 1", site_lat, site_lon, cam1_azimuths, fov, dist_km, cam1_colors)
add_camera_cones(map_obj, "Cam 2", site_lat, site_lon, cam2_azimuths, fov, dist_km, cam2_colors)

In [11]:
# Display the map
map_obj